In [3]:
#read in the data
import pandas as pd
import requests
import urllib.parse
import time

df2025=pd.read_csv("dfc_2025_geocoded.csv")
df2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7556 entries, 0 to 7555
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CCN                               7556 non-null   int64  
 1   Network                           7556 non-null   int64  
 2   Facility Name                     7556 non-null   object 
 3   Five Star Date                    7556 non-null   object 
 4   Five Star                         7013 non-null   float64
 5   Five Star Data Availability Code  7556 non-null   int64  
 6   Address Line 1                    7556 non-null   object 
 7   Address Line 2                    1262 non-null   object 
 8   City                              7556 non-null   object 
 9   State                             7556 non-null   object 
 10  Zip                               7556 non-null   int64  
 11  County/Parish                     7556 non-null   object 
 12  dfc_ne

In [ ]:
#time to tidy

#remove white space
df2025.columns = df2025.columns.str.strip()

df2025.rename(columns={"CMS Certification Number (CCN)": "CCN"}, inplace=True)
df2025.rename(columns={"City/Town": "City"}, inplace=True)
df2025.rename(columns={"ZIP Code": "Zip"}, inplace=True)
df2025.rename(columns={"County/Parish": "County"}, inplace=True)

#make all lowercase

df2025.columns = df2025.columns.str.lower()

#make facility names lowercase
df2025['facility name'] = df2025['facility name'].str.lower()

##verify
df2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7556 entries, 0 to 7555
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ccn                               7556 non-null   int64  
 1   network                           7556 non-null   int64  
 2   facility name                     7556 non-null   object 
 3   five star date                    7556 non-null   object 
 4   five star                         7013 non-null   float64
 5   five star data availability code  7556 non-null   int64  
 6   address line 1                    7556 non-null   object 
 7   address line 2                    1262 non-null   object 
 8   city                              7556 non-null   object 
 9   state                             7556 non-null   object 
 10  zip                               7556 non-null   int64  
 11  county                            7556 non-null   object 
 12  dfc_ne

In [5]:
#Function to Get Tract from Coordinates

def get_census_tract(latitude, longitude):
    base_url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
    params = {
        "x": longitude,
        "y": latitude,
        "benchmark": "Public_AR_Census2020",
        "vintage": "Census2020_Census2020",
        "format": "json"
    }
    response = requests.get(base_url, params=params)
    try:
        data = response.json()
        geos = data['result']['geographies']['Census Tracts'][0]
        return geos['GEOID']  # This is the full 11-digit tract FIPS code
    except (IndexError, KeyError):
        return None

In [8]:
#apply to my dataframe
tracts = []
for idx, row in df2025.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    tract = get_census_tract(latitude, longitude)
    tracts.append(tract)
    time.sleep(0.25)  # Sleep to avoid rate-limiting and be nice to API

# Add result to DataFrame
df2025['census_tract'] = tracts

df2025.to_csv("df2025_with_tracts.csv", index=False)